In [1]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel
import jieba
import opencc
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

# 使用Hugging Face提供的标准中文GPT-2模型
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 创建繁简转换器
cc = opencc.OpenCC('s2t')  # s2t 是 Simplified to Traditional 的简写

# 定义生成段落的函数
def generate_paragraph(prompt, max_length=600):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)  # 设置 attention_mask
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    traditional_text = cc.convert(text)  # 转换为繁体中文
    return traditional_text

# 生成20个文本
def generate_multiple_texts(prompt, num_texts=20):
    texts = []
    for i in range(num_texts):
        text = generate_paragraph(prompt)
        texts.append(text)
    return texts

prompt = "寫一篇關於外星人的幻想文章："
texts = generate_multiple_texts(prompt, num_texts=20)


/Users/chenhongyan/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /uer/gpt2-chinese-cluecorpussmall/resolve/main/model.safetensors (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fb90ae4de70>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 09e775ea-2956-4431-8647-8eec44453475)')

In [ ]:
# 计算困惑度
def calculate_perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(model.device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

# 计算BLEU分数
def calculate_bleu(reference_text, generated_text):
    reference = [reference_text.split()]
    candidate = generated_text.split()
    score = sentence_bleu(reference, candidate)
    return score

# 计算ROUGE分数
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, generated_text)
    return scores['rouge1'].fmeasure, scores['rougeL'].fmeasure

# 计算重复率
def calculate_repetition(text):
    words = text.split()
    word_count = Counter(words)
    total_words = len(words)
    repeated_words = sum(count for word, count in word_count.items() if count > 1)
    repetition_rate = repeated_words / total_words
    return repetition_rate

# 对所有生成的文本进行评分
def score_texts(texts, reference_text):
    scores = []
    for text in texts:
        perplexity = calculate_perplexity(model, tokenizer, text)
        bleu = calculate_bleu(reference_text, text)
        rouge1, rougeL = calculate_rouge(reference_text, text)
        repetition_rate = calculate_repetition(text)
        
        scores.append({
            'text': text,
            'perplexity': perplexity,
            'bleu': bleu,
            'rouge1': rouge1,
            'rougeL': rougeL,
            'repetition_rate': repetition_rate
        })
    return scores

# 示例参考文本（可以用你期望的文本替换）
reference_text = "這是一篇關於外星人的幻想文章，講述了人類與外星生物之間的接觸和衝突。"

# 对生成的20个文本进行评分
scores = score_texts(texts, reference_text)


In [ ]:
# 根据不同的指标挑选最佳文本
def select_best_texts(scores, top_n=5):
    # 可以根据不同的指标进行排序
    sorted_by_bleu = sorted(scores, key=lambda x: x['bleu'], reverse=True)
    sorted_by_rouge = sorted(scores, key=lambda x: x['rouge1'], reverse=True)
    sorted_by_perplexity = sorted(scores, key=lambda x: x['perplexity'])
    
    # 选择前 n 个得分最高的文本
    top_texts_by_bleu = sorted_by_bleu[:top_n]
    top_texts_by_rouge = sorted_by_rouge[:top_n]
    top_texts_by_perplexity = sorted_by_perplexity[:top_n]

    return top_texts_by_bleu, top_texts_by_rouge, top_texts_by_perplexity

# 挑选前5个文本
top_texts_by_bleu, top_texts_by_rouge, top_texts_by_perplexity = select_best_texts(scores, top_n=5)


In [ ]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel
import jieba
import opencc
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

# 使用Hugging Face提供的标准中文GPT-2模型
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 创建繁简转换器
cc = opencc.OpenCC('s2t')  # s2t 是 Simplified to Traditional 的简写

# 定义生成段落的函数
def generate_paragraph(prompt, max_length=600):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)  # 设置 attention_mask
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    traditional_text = cc.convert(text)  # 转换为繁体中文
    return traditional_text

# 生成20个文本
def generate_multiple_texts(prompt, num_texts=20):
    texts = []
    for i in range(num_texts):
        text = generate_paragraph(prompt)
        texts.append(text)
    return texts

prompt = "寫一篇關於外星人的幻想文章："
texts = generate_multiple_texts(prompt, num_texts=20)

# 计算困惑度
def calculate_perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(model.device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

# 计算BLEU分数
def calculate_bleu(reference_text, generated_text):
    reference = [reference_text.split()]
    candidate = generated_text.split()
    score = sentence_bleu(reference, candidate)
    return score

# 计算ROUGE分数
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, generated_text)
    return scores['rouge1'].fmeasure, scores['rougeL'].fmeasure

# 计算重复率
def calculate_repetition(text):
    words = text.split()
    word_count = Counter(words)
    total_words = len(words)
    repeated_words = sum(count for word, count in word_count.items() if count > 1)
    repetition_rate = repeated_words / total_words
    return repetition_rate

# 对所有生成的文本进行评分
def score_texts(texts, reference_text):
    scores = []
    for text in texts:
        perplexity = calculate_perplexity(model, tokenizer, text)
        bleu = calculate_bleu(reference_text, text)
        rouge1, rougeL = calculate_rouge(reference_text, text)
        repetition_rate = calculate_repetition(text)
        
        scores.append({
            'text': text,
            'perplexity': perplexity,
            'bleu': bleu,
            'rouge1': rouge1,
            'rougeL': rougeL,
            'repetition_rate': repetition_rate
        })
    return scores

# 示例参考文本（可以用你期望的文本替换）
reference_text = "這是一篇關於外星人的幻想文章，講述了人類與外星生物之間的接觸和衝突。"

# 对生成的20个文本进行评分
scores = score_texts(texts, reference_text)

# 根据不同的指标挑选最佳文本
def select_best_texts(scores, top_n=5):
    # 可以根据不同的指标进行排序
    sorted_by_bleu = sorted(scores, key=lambda x: x['bleu'], reverse=True)
    sorted_by_rouge = sorted(scores, key=lambda x: x['rouge1'], reverse=True)
    sorted_by_perplexity = sorted(scores, key=lambda x: x['perplexity'])
    
    # 选择前 n 个得分最高的文本
    top_texts_by_bleu = sorted_by_bleu[:top_n]
    top_texts_by_rouge = sorted_by_rouge[:top_n]
    top_texts_by_perplexity = sorted_by_perplexity[:top_n]

    return top_texts_by_bleu, top_texts_by_rouge, top_texts_by_perplexity

# 挑选前5个文本并显示
top_texts_by_bleu, top_texts_by_rouge, top_texts_by_perplexity = select_best_texts(scores, top_n=5)

# 打印挑选出的文本及其评分
def print_best_texts(texts, title):
    print(f"\n=== {title} ===")
    for idx, entry in enumerate(texts):
        print(f"\nText {idx + 1}:")
        print(f"Perplexity: {entry['perplexity']}")
        print(f"BLEU: {entry['bleu']}")
        print(f"ROUGE-1: {entry['rouge1']}")
        print(f"ROUGE-L: {entry['rougeL']}")
        print(f"Repetition Rate: {entry['repetition_rate']}")
        print(f"Text:\n{entry['text']}")

print_best_texts(top_texts_by_bleu, "Top Texts by BLEU Score")
print_best_texts(top_texts_by_rouge, "Top Texts by ROUGE Score")
print_best_texts(top_texts_by_perplexity, "Top Texts by Perplexity")


/Users/chenhongyan/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== Top Texts by BLEU Score ===

Text 1:
Perplexity: 8.262715339660645
BLEU: 0
ROUGE-1: 0.0
ROUGE-L: 0
Repetition Rate: 0.5971479500891266
Text:
寫 一 篇 關 於 外 星 人 的 幻 想 文 章 ：? 我 們 在 地 球 上 ， 有 很 多 人 ， 他 們 都 是 外 來 者 。 但 是 ， 我 們 也 可 以 通 過 這 些 人 來 了 解 到 ， 外 來 者 的 存 在 。 比 如 說 ， 你 看 到 一 個 外 國 人 在 那 裡 ， 然 後 你 就 會 發 現 ， 原 來 他 們 是 在 做 一 件 事 情 。 而 且 ， 這 種 行 為 並 不 是 單 純 的 外 語 ， 而 是 一 種 思 維 方 式 。 所 以 ， 當 你 認 識 到 外 面 的 世 界 ， 就 能 夠 理 解 這 個 世 間 的 真 實 。 因 此 ， 對 於 這 樣 的 人 或 物 ， 要 學 習 其 中 的 一 部 分 。 這 就 是 我 的 觀 點 。 我 覺 得 ， 在 這 裏 ， 大 家 應 該 明 白 ， 人 類 的 生 活 ， 是 由 外 界 引 導 的 。 外 表 的 美 麗 ， 只 是 人 類 的 本 性 。 人 們 的 心 靈 ， 才 是 最 重 要 的 ， 它 需 要 外 在 的 力 量 去 支 持 和 激 勵 。 內 容 摘 自 網 絡 ， 侵 刪 。 - - end - - 作 者 ： 李 小 萌 （ 微 信 id : lieqiuzi ） ， 著 名 科 學 家 、 教 育 專 欄 作 家 ， 科 技 博 客 作 品 《 科 普 漫 畫 》 創 始 人 。 科 研 成 果 轉 載 請 注 明 出 處 。 更 多 精 彩 內 容 ， 歡 迎 關 注 科 大 訊 飛 官 方 微 博 ： @ 科 訊 聲 明 ： 科 爾 兄 弟 原 創 文 字 ， 未 經 授 權 禁 止 轉 發 ！ 合 作 聯 系 qq ： 644554782 編 輯 ： 王 曉 丹 投 稿 郵 箱 ： kzjrbx @ 163. com 交 流 羣 ： 464272181 報 道 熱 線 ： 400 - 656 - 568 務 必 加 入 哦 ~ 

In [2]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel
import jieba
import opencc
import random
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from bert_score import score as bert_score
from concurrent.futures import ThreadPoolExecutor, as_completed

# 设置日志记录
import logging
logging.basicConfig(level=logging.INFO)

# 使用标准中文GPT-2模型
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 创建繁简转换器
cc = opencc.OpenCC('s2t')

# 定义生成段落的函数
def generate_paragraph(prompt, max_length=600):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.85,
        top_k=30,
        top_p=0.9,
        repetition_penalty=1.7,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    traditional_text = cc.convert(text)
    return traditional_text

# 生成多个文本并进行多样化
def generate_multiple_texts(prompt, num_texts=100):
    texts = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(generate_paragraph, prompt + " " + random.choice(["", "故事開始", "另一個視角", "新情節"])) for _ in range(num_texts)]
        for future in as_completed(futures):
            texts.append(future.result())
    return texts

# 计算困惑度
def calculate_perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(model.device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

# 计算BLEU分数
def calculate_bleu(reference_text, generated_text):
    reference = [reference_text.split()]
    candidate = generated_text.split()
    score = sentence_bleu(reference, candidate)
    return score

# 计算ROUGE分数
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, generated_text)
    return scores['rouge1'].fmeasure, scores['rougeL'].fmeasure

# 计算BERTScore
def calculate_bertscore(reference_text, generated_text):
    P, R, F1 = bert_score([generated_text], [reference_text], lang='zh', verbose=True)
    return F1.mean().item()

# 计算重复率
def calculate_repetition(text):
    words = text.split()
    word_count = Counter(words)
    total_words = len(words)
    repeated_words = sum(count for word, count in word_count.items() if count > 1)
    repetition_rate = repeated_words / total_words
    return repetition_rate

# 对所有生成的文本进行评分
def score_texts(texts, reference_text):
    scores = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        for text in texts:
            futures.append(executor.submit(score_text, text, reference_text))
        for future in as_completed(futures):
            scores.append(future.result())
    return scores

# 单独评分函数
def score_text(text, reference_text):
    try:
        perplexity = calculate_perplexity(model, tokenizer, text)
        bleu = calculate_bleu(reference_text, text)
        rouge1, rougeL = calculate_rouge(reference_text, text)
        bertscore = calculate_bertscore(reference_text, text)
        repetition_rate = calculate_repetition(text)

        return {
            'text': text,
            'perplexity': perplexity,
            'bleu': bleu,
            'rouge1': rouge1,
            'rougeL': rougeL,
            'bertscore': bertscore,
            'repetition_rate': repetition_rate
        }
    except Exception as e:
        logging.error(f"Error scoring text: {e}")
        return None

# 示例参考文本
reference_text = "這是一篇關於外星人的幻想文章，講述了人類與外星生物之間的接觸和衝突。"

# 对生成的20个文本进行评分
prompt = "寫一篇關於外星人的幻想文章："
texts = generate_multiple_texts(prompt, num_texts=20)
scores = score_texts(texts, reference_text)

# 挑选出评分最高的文本
best_text = max(scores, key=lambda x: x['bertscore'] if x else -1)

# 输出每个文本的分数
for i, score in enumerate(scores):
    if score:
        print(f"\nText {i + 1}:")
        print(f"Perplexity: {score['perplexity']}")
        print(f"BLEU: {score['bleu']}")
        print(f"ROUGE-1: {score['rouge1']}")
        print(f"ROUGE-L: {score['rougeL']}")
        print(f"BERTScore: {score['bertscore']}")
        print(f"Repetition Rate: {score['repetition_rate']}")
        print(f"Text:\n{score['text']}\n")

# 输出最佳文本
if best_text:
    print("\n=== Best Text by BERTScore ===")
    print(f"Perplexity: {best_text['perplexity']}")
    print(f"BLEU: {best_text['bleu']}")
    print(f"ROUGE-1: {best_text['rouge1']}")
    print(f"ROUGE-L: {best_text['rougeL']}")
    print(f"BERTScore: {best_text['bertscore']}")
    print(f"Repetition Rate: {best_text['repetition_rate']}")
    print(f"Text:\n{best_text['text']}\n")

    # 将最佳文本保存为txt文件
    with open("best_generated_text.txt", "w", encoding="utf-8") as file:
        file.write(best_text['text'])

    print("最佳文本已保存到 best_generated_text.txt 文件中")
else:
    print("No valid texts were generated or scored.")


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /uer/gpt2-chinese-cluecorpussmall/resolve/main/model.safetensors (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fb90be1c8b0>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 574952ff-8f16-4fa3-9440-0da447d518a3)')

In [3]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel
import jieba
import opencc
import random
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from bert_score import score as bert_score
from concurrent.futures import ThreadPoolExecutor, as_completed

# 設置日誌記錄
import logging
logging.basicConfig(level=logging.INFO)

# 使用標準中文GPT-2模型
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

# 將模型移動到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 創建繁簡轉換器
cc = opencc.OpenCC('s2t')  # s2t 是 Simplified to Traditional 的簡寫

# 定義生成段落的函數
def generate_paragraph(prompt, max_length=600):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.85,
        top_k=30,
        top_p=0.9,
        repetition_penalty=1.7,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    traditional_text = cc.convert(text)
    return format_text(traditional_text)

# 格式化文本，每50個字換行
def format_text(text):
    formatted_text = ""
    current_line = ""
    count = 0
    for char in text:
        current_line += char
        count += 1
        if count >= 50:
            formatted_text += current_line + "\n"
            current_line = ""
            count = 0
    if current_line:
        formatted_text += current_line
    return formatted_text

# 生成多個文本並進行多樣化
def generate_multiple_texts(prompt, num_texts=100):
    texts = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(generate_paragraph, prompt + " " + random.choice(["", "故事開始", "另一個視角", "新情節"])) for _ in range(num_texts)]
        for future in as_completed(futures):
            texts.append(future.result())
    return texts

# 計算困惑度
def calculate_perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors='pt')
    max_length = model.config.n_positions
    stride = 512

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(model.device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

# 計算BLEU分數
def calculate_bleu(reference_text, generated_text):
    reference = [reference_text.split()]
    candidate = generated_text.split()
    score = sentence_bleu(reference, candidate)
    return score

# 計算ROUGE分數
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, generated_text)
    return scores['rouge1'].fmeasure, scores['rougeL'].fmeasure

# 計算BERTScore
def calculate_bertscore(reference_text, generated_text):
    P, R, F1 = bert_score([generated_text], [reference_text], lang='zh', verbose=True)
    return F1.mean().item()

# 計算重複率
def calculate_repetition(text):
    words = text.split()
    word_count = Counter(words)
    total_words = len(words)
    repeated_words = sum(count for word, count in word_count.items() if count > 1)
    repetition_rate = repeated_words / total_words
    return repetition_rate

# 對所有生成的文本進行評分
def score_texts(texts, reference_text):
    scores = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        for text in texts:
            futures.append(executor.submit(score_text, text, reference_text))
        for future in as_completed(futures):
            scores.append(future.result())
    return scores

# 單獨評分函數
def score_text(text, reference_text):
    try:
        perplexity = calculate_perplexity(model, tokenizer, text)
        bleu = calculate_bleu(reference_text, text)
        rouge1, rougeL = calculate_rouge(reference_text, text)
        bertscore = calculate_bertscore(reference_text, text)
        repetition_rate = calculate_repetition(text)

        return {
            'text': text,
            'perplexity': perplexity,
            'bleu': bleu,
            'rouge1': rouge1,
            'rougeL': rougeL,
            'bertscore': bertscore,
            'repetition_rate': repetition_rate
        }
    except Exception as e:
        logging.error(f"Error scoring text: {e}")
        return None

# 示例參考文本
reference_text = "這是一篇關於外星人的幻想文章，講述了人類與外星生物之間的接觸和衝突。"

# 對生成的20個文本進行評分
prompt = "寫一篇關於外星人的幻想文章："
texts = generate_multiple_texts(prompt, num_texts=20)
scores = score_texts(texts, reference_text)

# 挑選出評分最高的文本
best_text = max(scores, key=lambda x: x['bertscore'] if x else -1)

# 輸出每個文本的分數
for i, score in enumerate(scores):
    if score:
        print(f"\nText {i + 1}:")
        print(f"Perplexity: {score['perplexity']}")
        print(f"BLEU: {score['bleu']}")
        print(f"ROUGE-1: {score['rouge1']}")
        print(f"ROUGE-L: {score['rougeL']}")
        print(f"BERTScore: {score['bertscore']}")
        print(f"Repetition Rate: {score['repetition_rate']}")
        print(f"Text:\n{score['text']}\n")

# 輸出最佳文本
if best_text:
    print("\n=== Best Text by BERTScore ===")
    print(f"Perplexity: {best_text['perplexity']}")
    print(f"BLEU: {best_text['bleu']}")
    print(f"ROUGE-1: {best_text['rouge1']}")
    print(f"ROUGE-L: {best_text['rougeL']}")
    print(f"BERTScore: {best_text['bertscore']}")
    print(f"Repetition Rate: {best_text['repetition_rate']}")
    print(f"Text:\n{best_text['text']}\n")

    # 將最佳文本保存為txt文件
    with open("best_generated_text.txt", "w", encoding="utf-8") as file:
        file.write(best_text['text'])

    print("最佳文本已保存到 best_generated_text.txt 文件中")
else:
    print("No valid texts were generated or scored.")


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /uer/gpt2-chinese-cluecorpussmall/resolve/main/model.safetensors (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fb8fc150b50>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 0d11b993-c3a2-49d8-8669-fc4dfafe8934)')

In [5]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel
import opencc
import random
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from collections import Counter
from bert_score import score as bert_score
from concurrent.futures import ThreadPoolExecutor, as_completed

# 設置日誌記錄
import logging
logging.basicConfig(level=logging.INFO)

# 使用標準中文GPT-2模型
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

# 將模型移動到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 創建繁簡轉換器
cc = opencc.OpenCC('s2t')  # s2t 是 Simplified to Traditional 的簡寫

# 定義生成段落的函數
def generate_paragraph(prompt, max_length=600, temperature=0.85, top_k=30, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=1.7,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    traditional_text = cc.convert(text)
    return format_text(traditional_text)

# 格式化文本，每50個字換行
def format_text(text):
    formatted_text = ""
    current_line = ""
    count = 0
    for char in text:
        current_line += char
        count += 1
        if count >= 50:
            formatted_text += current_line + "\n"
            current_line = ""
            count = 0
    if current_line:
        formatted_text += current_line
    return formatted_text

# 測試不同的參數組合
def test_parameters(prompt, num_texts=100, temperature_list=[0.7, 0.85, 1.0], top_k_list=[30, 50], top_p_list=[0.8, 0.9, 1.0]):
    best_text = None
    best_score = -float('inf')
    best_params = {}

    for temperature in temperature_list:
        for top_k in top_k_list:
            for top_p in top_p_list:
                logging.info(f"Testing with temperature={temperature}, top_k={top_k}, top_p={top_p}")
                texts = generate_multiple_texts(prompt, num_texts, temperature, top_k, top_p)
                scores = score_texts(texts, reference_text)
                top_text = max(scores, key=lambda x: x['bertscore'] if x else -1)
                if top_text and top_text['bertscore'] > best_score:
                    best_score = top_text['bertscore']
                    best_text = top_text
                    best_params = {
                        'temperature': temperature,
                        'top_k': top_k,
                        'top_p': top_p
                    }

    return best_text, best_params

# 生成多個文本並進行多樣化
def generate_multiple_texts(prompt, num_texts=100, temperature=0.85, top_k=30, top_p=0.9):
    texts = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(generate_paragraph, prompt + " " + random.choice(["", "故事開始", "另一個視角", "新情節"]),
                                   600, temperature, top_k, top_p) for _ in range(num_texts)]
        for future in as_completed(futures):
            texts.append(future.result())
    return texts

# 計算BLEU分數
def calculate_bleu(reference_text, generated_text):
    reference = [reference_text.split()]
    candidate = generated_text.split()
    score = sentence_bleu(reference, candidate)
    return score

# 計算ROUGE分數
def calculate_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, generated_text)
    return scores['rouge1'].fmeasure, scores['rougeL'].fmeasure

# 計算BERTScore
def calculate_bertscore(reference_text, generated_text):
    P, R, F1 = bert_score([generated_text], [reference_text], lang='zh', verbose=True)
    return F1.mean().item()

# 計算重複率
def calculate_repetition(text):
    words = text.split()
    word_count = Counter(words)
    total_words = len(words)
    repeated_words = sum(count for word, count in word_count.items() if count > 1)
    repetition_rate = repeated_words / total_words
    return repetition_rate

# 單獨評分函數
def score_text(text, reference_text):
    try:
        bleu = calculate_bleu(reference_text, text)
        rouge1, rougeL = calculate_rouge(reference_text, text)
        bertscore = calculate_bertscore(reference_text, text)
        repetition_rate = calculate_repetition(text)

        return {
            'text': text,
            'bleu': bleu,
            'rouge1': rouge1,
            'rougeL': rougeL,
            'bertscore': bertscore,
            'repetition_rate': repetition_rate
        }
    except Exception as e:
        logging.error(f"Error scoring text: {e}")
        return None

# 對所有生成的文本進行評分
def score_texts(texts, reference_text):
    scores = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(score_text, text, reference_text) for text in texts]
        for future in as_completed(futures):
            result = future.result()
            if result:
                scores.append(result)
    return scores

# 示例參考文本
reference_text = "這是一篇關於外星人的幻想文章，講述了人類與外星生物之間的接觸和衝突。"

# 測試不同參數組合並生成文本
prompt = "寫一篇關於外星人的幻想文章："
best_text, best_params = test_parameters(prompt, num_texts=100)

# 輸出最佳結果
if best_text:
    print("\n=== Best Text ===")
    print(f"Best Parameters: {best_params}")
    print(f"Text:\n{best_text['text']}")

    # 將最佳文本保存為txt文件
    with open("best_generated_text_with_params.txt", "w", encoding="utf-8") as file:
        file.write(best_text['text'])
    print("最佳文本已保存到 best_generated_text_with_params.txt 文件中")
else:
    print("No valid texts were generated or scored.")


/Users/chenhongyan/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/Users/chenhongyan/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:root:Testing with temperature=0.7, top_k=30, top_p=0.8
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/chenhongy

calculating scores...
computing bert embedding.
calculating scores...
computing bert embedding.
calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:04<00:00,  4.17s/it]


computing greedy matching.


100%|██████████| 1/1 [00:04<00:00,  4.23s/it]


...

computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 71.04it/s]



100%|██████████| 1/1 [00:03<00:00,  3.40s/it]


done in 3.61 seconds, 0.28 sentences/sec
computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 229.77it/s]

done in 3.42 seconds, 0.29 sentences/sec

=== Best Text ===
Best Parameters: {'temperature': 0.7, 'top_k': 30, 'top_p': 0.8}
Text:
寫 一 篇 關 於 外 星 人 的 幻 想 文 章 ： 另 一 個 視 角 我 們 在 地 球 上 
生 活 了 很 多 年 ， 但 是 有 些 事 情 不 能 忘 記 。 比 如 說 你 看 到 過 
太 陽 和 月 亮 嗎 ？ 那 麼 這 兩 者 之 間 會 發 展 出 什 麼 樣 子 呢? 答 案
 就 是 宇 宙 中 存 留 著 許 許 多 多 未 知 物 種 、 甚 至 可 以 用 來 形 容
 它 們 的 現 象.. ． 所 謂 「 無 限 」 或 稱 為 「 無 意 識 」 ( ), 即 
指 任 何 時 候 都 沒 法 去 理 解 其 他 東 西 而 已 。 因 此 ， 只 要 把 這 
個 世 界 觀 放 置 於 自 己 身 邊 便 會 感 受 到 真 正 的 美 好 與 快 樂! 然
 後 再 回 頭 重 新 思 考 下 面 幾 點 問 題 : 1 ） 假 設 某 天 妳 遇 見 了
 一 位 神 祕 力 量 ── － - - 超 越 現 實 主 義 者 ──── 祂 （ 她 ／ 她
 / 她 ） 擁 抱 着 巨 大 的 光 芒 與 熱 忱 ； 2 ） 倘 若 妳 碰 見 了 同 樣
 具 備 相 似 特 質 且 又 互 爲 影 響 力 的 科 學 家 及 工 程 師 ﹔ 3 ） 妳
 將 永 遠 成 長 並 改 變 妳 的 命 運 ; 4 ） 當 妳 遭 遇 困 境 時 ﹐ 牠 必
 須 努 力 克 服 並 尋 求 救 援 方 式 來 幫 助 妳 渡 過 難 關 。 5 ） 當 妳
 被 迫 接 近 失 敗 後 ﹐ 將 對 於 挫 折 產 生 恐 懼 心 態 ︰ ①②③④⑤⑥① ②
 ③ ④ ⑤ ⑥ ⑦ ⑧ ⑨ ⑩⑦⑧⑨⑩ ★☆☆no3★☆1 、 首 先 您 需 要 明 白 ， 每
 次 發 射 火 箭 前 ， 最 初 的 目 標 應 該 是 怎 麼 定 向 飛 行 器 起 降 。
 由 於 火 焰 燃 燒 產 生 熱 氣 體 ， 使 得 空 間 環 流 加 速 度 減 慢 ， 從
 而